In [1]:
import requests
import pandas as pd
import numpy as np

Some test ids

In [2]:
tf2_id = 440
csgo_id = 730
rl_id = 252950
fo4_id = 377160
starfield_id = 1716740

Function to build a row for the dataframe based on the game's avaialble information

In [3]:
def fetch(game_info, categories):
    info = {}
    for category in categories:
        cur_info = game_info
        failed = False
        
        for subpart in category:
            if not failed and subpart in cur_info.keys():
                cur_info = cur_info[subpart]
            else:
                failed = True
        
        if not failed:
            info[' '.join(category)] = cur_info
        else:
            info[' '.join(category)] = np.NaN
    
    # Genre is handled differently from the rest because a game can have multiple genres.
    # They can also have multiple publishers/developers but I don't think that we should
    # treat that the same way, because the numbers of genres should be much less than the
    # numbers of developers.
    # Not sure how the "dummy variable trap" works here either. I think each row should
    # have a column for each genre, but it doesn't make sense for the 0 vector to default
    # to some genre when games can have more than one. For example, if the 0 vector defaulted
    # to FPS, then for a game that is an FPS and RPG, what columns should be marked 1? If
    # it is just the RPG column, then how would you tell which games are just RPGs and which
    # ones are FPS RPG hybrids.
    # Also could use 'categories' instead of 'genres', but 'categories' has a bunch of extra
    # stuff like if the game has steam achievements etc.
    # Could also do both, but that would be a lot of columns (probably will be a bunch with
    # just genres)
    if 'genres' in game_info.keys():
        for genre in game_info['genres']:
            info['Genre: ' + genre['description']] = True
    
    
    return info

Categories to go into the dataframe

In [4]:
categories = [
    ['name'],
    ['type'],
    ['steam_appid'],
    ['developers'],
    ['publishers'],
    ['is_free'],
    ['price_overview', 'initial'], # initial doesn't mean price on launch, it means the price right now excluding sales (at least I'm 90% sure)
    ['achievements', 'total'],
    ['release_date', 'date'],
    ['metacritic', 'score']
    # ['recommendations', 'total'] unneccessary with reviews api
]

filters = ''
for cat in categories:
    filters += cat[0] + ','
filters += 'basic,genres'

Iterates through listed apps and stores the data\
Potential issue here is that sometimes the information CSGO returns is in Russian. Sometimes it is in English. I didn't see any parameter to specify the result language, so I'm not quite sure what to do about this.\
I may have fixed the issue by adding this filters thing as a parameter? Because the filters are in English.

In [5]:
all_info = []
app_ids = [tf2_id, fo4_id, rl_id, csgo_id, starfield_id]

for app_id in app_ids:
    # make main api query
    URL = "http://store.steampowered.com/api/appdetails"
    PARAMS = {'appids': app_id, 'filters': filters}
    re = requests.get(url = URL, params = PARAMS)
    game_info = re.json()[str(app_id)]['data']
    info = fetch(game_info, categories)
    # make review api query
    URL = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
    PARAMS = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
    re = requests.get(url = URL, params = PARAMS)
    summary = re.json()
    if 'query_summary' in summary.keys():
        q_summary = summary['query_summary']
        
        if 'total_positive' in q_summary:
            info['total_positive'] = q_summary['total_positive']
        else:
            info['total_positive'] = np.NaN
            
        if 'total_reviews' in q_summary:
            info['total_reviews'] = q_summary['total_reviews']
        else:
            info['total_reviews'] = np.NaN
    
    
    all_info.append(info)

Creates a dataframe from the data

In [6]:
df = pd.DataFrame.from_dict(all_info)
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,Genre: Action,Genre: Free to Play,total_positive,total_reviews,Genre: RPG,Genre: Indie,Genre: Racing,Genre: Sports
0,Team Fortress 2,game,440,[Valve],[Valve],True,NaN,520.0,"Oct 10, 2007",92.0,True,True,806229,861234,NaN,NaN,NaN,NaN
1,Fallout 4,game,377160,[Bethesda Game Studios],[Bethesda Softworks],False,1999.0,84.0,"Nov 9, 2015",84.0,NaN,NaN,223442,276245,True,NaN,NaN,NaN
2,Rocket League®,game,252950,[Psyonix LLC],[Psyonix LLC],False,NaN,88.0,"Jul 6, 2015",86.0,True,NaN,487965,544390,NaN,True,True,True
3,Counter-Strike: Global Offensive,game,730,"[Valve, Hidden Path Entertainment]",[Valve],True,NaN,167.0,"Aug 21, 2012",83.0,True,True,5621135,6371049,NaN,NaN,NaN,NaN
4,Starfield,game,1716740,[Bethesda Game Studios],[Bethesda Softworks],False,NaN,NaN,11.11.22,NaN,NaN,NaN,0,0,True,NaN,NaN,NaN


Replaces NaN values from the genre columns with false instead.

In [7]:
for col_name in df:
    if col_name.startswith('Genre: '):
        df[col_name].fillna(False, inplace=True)
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,Genre: Action,Genre: Free to Play,total_positive,total_reviews,Genre: RPG,Genre: Indie,Genre: Racing,Genre: Sports
0,Team Fortress 2,game,440,[Valve],[Valve],True,NaN,520.0,"Oct 10, 2007",92.0,True,True,806229,861234,False,False,False,False
1,Fallout 4,game,377160,[Bethesda Game Studios],[Bethesda Softworks],False,1999.0,84.0,"Nov 9, 2015",84.0,False,False,223442,276245,True,False,False,False
2,Rocket League®,game,252950,[Psyonix LLC],[Psyonix LLC],False,NaN,88.0,"Jul 6, 2015",86.0,True,False,487965,544390,False,True,True,True
3,Counter-Strike: Global Offensive,game,730,"[Valve, Hidden Path Entertainment]",[Valve],True,NaN,167.0,"Aug 21, 2012",83.0,True,True,5621135,6371049,False,False,False,False
4,Starfield,game,1716740,[Bethesda Game Studios],[Bethesda Softworks],False,NaN,NaN,11.11.22,NaN,False,False,0,0,True,False,False,False


Fixes the release dates. The parameter `errors='coerce'` means that if pandas cannot figure out what the date format means, then it will set the value to pd.NaT. This could be important if the date gets returned in another language.

In [8]:
df['release_date date'] = df['release_date date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,Genre: Action,Genre: Free to Play,total_positive,total_reviews,Genre: RPG,Genre: Indie,Genre: Racing,Genre: Sports
0,Team Fortress 2,game,440,[Valve],[Valve],True,NaN,520.0,2007-10-10,92.0,True,True,806229,861234,False,False,False,False
1,Fallout 4,game,377160,[Bethesda Game Studios],[Bethesda Softworks],False,1999.0,84.0,2015-11-09,84.0,False,False,223442,276245,True,False,False,False
2,Rocket League®,game,252950,[Psyonix LLC],[Psyonix LLC],False,NaN,88.0,2015-07-06,86.0,True,False,487965,544390,False,True,True,True
3,Counter-Strike: Global Offensive,game,730,"[Valve, Hidden Path Entertainment]",[Valve],True,NaN,167.0,2012-08-21,83.0,True,True,5621135,6371049,False,False,False,False
4,Starfield,game,1716740,[Bethesda Game Studios],[Bethesda Softworks],False,NaN,NaN,2022-11-11,NaN,False,False,0,0,True,False,False,False


Function to determine positive review percentage for a row. Gives NaN if missing either the total review count or total count of positive reviews.

In [9]:
def positive_perc(row):
    if not np.isnan(row['total_reviews']) and row['total_reviews'] != 0 and not np.isnan(row['total_positive']):
        return row['total_positive'] / row['total_reviews']
    else:
        return np.NaN

Adds positive review percentage column to the dataframe

In [10]:
df['positive_percentage'] = df.apply(positive_perc, axis=1)
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,Genre: Action,Genre: Free to Play,total_positive,total_reviews,Genre: RPG,Genre: Indie,Genre: Racing,Genre: Sports,positive_percentage
0,Team Fortress 2,game,440,[Valve],[Valve],True,NaN,520.0,2007-10-10,92.0,True,True,806229,861234,False,False,False,False,0.936132
1,Fallout 4,game,377160,[Bethesda Game Studios],[Bethesda Softworks],False,1999.0,84.0,2015-11-09,84.0,False,False,223442,276245,True,False,False,False,0.808854
2,Rocket League®,game,252950,[Psyonix LLC],[Psyonix LLC],False,NaN,88.0,2015-07-06,86.0,True,False,487965,544390,False,True,True,True,0.896352
3,Counter-Strike: Global Offensive,game,730,"[Valve, Hidden Path Entertainment]",[Valve],True,NaN,167.0,2012-08-21,83.0,True,True,5621135,6371049,False,False,False,False,0.882293
4,Starfield,game,1716740,[Bethesda Game Studios],[Bethesda Softworks],False,NaN,NaN,2022-11-11,NaN,False,False,0,0,True,False,False,False,NaN


Function to set price to 0 if the game is listed as free (most games just have NaN as price if they are free)

In [11]:
def get_price(row):
    if np.isnan(row['price_overview initial']) and row['is_free']:
        return 0
    else:
        return row['price_overview initial']

Apply the price changes and rename the column to just price

In [12]:
df['price_overview initial'] = df.apply(get_price, axis=1)
df.rename({'price_overview initial': 'price'}, axis='columns', inplace=True)
df.head()

,name,type,steam_appid,developers,publishers,is_free,price,achievements total,release_date date,metacritic score,Genre: Action,Genre: Free to Play,total_positive,total_reviews,Genre: RPG,Genre: Indie,Genre: Racing,Genre: Sports,positive_percentage
0,Team Fortress 2,game,440,[Valve],[Valve],True,0.0,520.0,2007-10-10,92.0,True,True,806229,861234,False,False,False,False,0.936132
1,Fallout 4,game,377160,[Bethesda Game Studios],[Bethesda Softworks],False,1999.0,84.0,2015-11-09,84.0,False,False,223442,276245,True,False,False,False,0.808854
2,Rocket League®,game,252950,[Psyonix LLC],[Psyonix LLC],False,NaN,88.0,2015-07-06,86.0,True,False,487965,544390,False,True,True,True,0.896352
3,Counter-Strike: Global Offensive,game,730,"[Valve, Hidden Path Entertainment]",[Valve],True,0.0,167.0,2012-08-21,83.0,True,True,5621135,6371049,False,False,False,False,0.882293
4,Starfield,game,1716740,[Bethesda Game Studios],[Bethesda Softworks],False,NaN,NaN,2022-11-11,NaN,False,False,0,0,True,False,False,False,NaN


Probably rearrange the columns so that the genres are all at the end, but that isn't super important for now